In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

# Find Pairs

In [2]:
#Data cleaning 
file_names_lst = ["DIA.csv", "FEZ.csv", "IJH.csv", "IWD.csv", "IWF.csv", "IYY.csv", 
                  "IWM.csv", "IWN.csv", "IWO.csv", "MDY.csv", "OEF.csv", "SPY.csv", "VTI.csv", "QQQ.csv",
                  "FTEC.csv", "IYW.csv", "VITAX.csv", "XLK.csv",
                  "XLV.csv", "IYH.csv", "VHT.csv", "XBI.csv", "IHI.csv",
                  "XLF.csv", "IYG.csv", "VFH.csv", "KIE.csv",
                  "XLE.csv", "IYE.csv", "VDE.csv", "XOP.csv"]
#Line 1 & 2: stock index #Line 3, Tech ETF. Line 4, Health Care ETF. #Line5, fin service. #Line6 engergy.
equity_names_lst = ["DIA", "FEZ", "IJH", "IWD", "IWF", "IYY",
                    "IWM", "IWN", "IWO", "MDY", "OEF", "QQQ", "SPY", "VTI",
                    "FTEC", "IYW", "VITAX", "XLK",
                    "XLV", "IYH", "VHT", "XBI", "IHI",
                    "XLF", "IYG", "VFH", "KIE",
                    "XLE", "IYE", "VDE", "XOP"]
uni_daily_prices = pd.DataFrame() # Dataframe with daily Adj Close
uni_daily_handl_prices = pd.DataFrame() # Dataframe with daily high and daily low
uni_daily_range = pd.DataFrame() # Dataframe with daily range(calculated by ln(high/low))
for i in range(len(file_names_lst)):
    file_name = file_names_lst[i]
    equity_name = equity_names_lst[i]
    df = pd.read_csv("Data/"+file_name)
    df = df[['Date', 'Adj Close', 'High', 'Low']]
    df['Daily Range'] = np.log(df["High"]/df["Low"])
    df['Date'] = pd.to_datetime(df['Date'])
    cutoff_date = pd.to_datetime('2007-01-01')
    df = df.loc[df['Date'] > cutoff_date, :]
    if i == 0:
        uni_daily_prices['Date'] = df['Date']
        uni_daily_handl_prices['Date'] = df['Date']
        uni_daily_range['Date'] = df['Date']
        uni_daily_prices.set_index('Date', inplace=True)
        uni_daily_handl_prices.set_index('Date', inplace=True)
        uni_daily_range.set_index('Date', inplace=True)
    df.set_index('Date', inplace=True)
    uni_daily_prices[equity_name] = df["Adj Close"]
    uni_daily_handl_prices[equity_name + " High"] = df["High"]
    uni_daily_handl_prices[equity_name + " Low"] = df["Low"]
    uni_daily_range[equity_name] = df["Daily Range"]
uni_daily_returns_df = uni_daily_prices.pct_change()
uni_monthly_prices = uni_daily_prices.resample('M').last()
uni_monthly_returns_df = uni_monthly_prices.pct_change()
uni_daily_returns_df = uni_daily_returns_df.iloc[1:]

In [3]:
uni_daily_returns_df.head()

,DIA,FEZ,IJH,IWD,IWF,IYY,IWM,IWN,IWO,MDY,...,XBI,IHI,XLF,IYG,VFH,KIE,XLE,IYE,VDE,XOP
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-04,0.001688,-0.000548,-0.001115,-0.002307,0.005618,0.002039,0.002687,0.001496,0.002924,0.001156,...,NaN,0.003151,-0.003250,-0.001491,-0.000457,NaN,-0.018205,-0.018872,-0.020548,-0.013100
2007-01-05,-0.007539,-0.017393,-0.011289,-0.007908,-0.006668,-0.008433,-0.020927,-0.020408,-0.020791,-0.011272,...,NaN,-0.003730,-0.007881,-0.006641,-0.010077,NaN,0.006660,0.006342,0.007617,0.009403
2007-01-08,0.003232,0.000559,0.001757,0.003434,0.004354,0.004399,-0.003909,0.000127,0.003107,0.000137,...,NaN,0.009852,0.004382,0.004057,0.003856,NaN,0.000000,0.002067,0.000620,0.004384
2007-01-09,-0.000564,-0.002421,0.004384,-0.002811,0.001265,0.000000,0.009420,0.003557,0.005550,0.005700,...,NaN,0.006439,-0.001636,-0.000973,-0.001690,NaN,-0.008583,-0.010207,-0.010652,-0.010366
2007-01-10,0.002902,-0.010827,0.005612,0.001348,0.006855,0.002189,0.007778,0.001898,0.004877,0.006896,...,NaN,0.002715,0.003278,0.001797,0.003386,NaN,-0.019300,-0.016875,-0.014021,-0.018743


In [4]:
def SSD(df):
    ssd_df = pd.DataFrame(index = ['SSD'])
    for i in range(len(df.columns)):
        for j in range(i+1, len(df.columns)):
            # calculate SSD between columns i and j
            col1 = df.iloc[:, i]
            col2 = df.iloc[:, j]
            ssd = ((col1 - col2) ** 2).sum()
            
            # store result in ssd_df
            colname = f"{df.columns[i]}-{df.columns[j]}"
            ssd_df[colname] = [ssd]
    
    return ssd_df

from scipy.stats import kendalltau

def Kendall_Tau(df):
    kendall_df = pd.DataFrame(index = ['KT'])
    # iterate over column pairs
    for i in range(len(df.columns)):
        for j in range(i+1, len(df.columns)):
            # calculate Kendall tau correlation between columns i and j
            col1 = df.iloc[:, i]
            col2 = df.iloc[:, j]
            tau, pval = kendalltau(col1, col2)
            # store result in kendall_df
            colname = f"{df.columns[i]}-{df.columns[j]}"
            kendall_df[colname] = [tau]
    # print results
    return kendall_df

# Select Pairs for General Stock indexes

In [5]:
names_l = ["DIA", "FEZ", "IJH", "IWD", "IWF", "IYY", "IWM", "IWN", "IWO", "MDY", "OEF", "QQQ", "SPY", "VTI",]
kt_daily_df = Kendall_Tau(uni_daily_returns_df[names_l]).T
max_kt_daily = kt_daily_df.idxmax(axis=0)[0]
kt_daily_df.sort_values(by='KT', ascending=False, inplace=True)

print(kt_daily_df[:10])
print(f"Maximum KT value(daily) is {kt_daily_df['KT'][max_kt_daily]} in column '{max_kt_daily}'")

               KT
IJH-MDY  0.954705
IYY-SPY  0.924421
QQQ-SPY  0.919160
IYY-QQQ  0.917013
OEF-QQQ  0.887501
IWM-IWO  0.872697
IWM-IWN  0.866647
OEF-SPY  0.863260
IYY-OEF  0.857171
IWF-SPY  0.827379
Maximum KT value(daily) is 0.9547053063498474 in column 'IJH-MDY'


## Selected:
IJH-MDY -- iShares Core S&P Mid-Cap ETF & SPDR S&P MIDCAP 400 ETF \
IYY-SPY -- iShares Dow Jones U.S. ETF & SPDR S&P 500 ETF Trust\
OEF-QQQ -- iShares S&P 100 ETF & Invesco Nasdaq-100 ETF\
IWM-IWO -- iShares Russell 1000 Value ETF & iShares Russell 2000 Growth ETF

# Select Pairs for Specific Industry ETFs

In [6]:
names_l = ["FTEC", "IYW", "VITAX", "XLK",
                    "XLV", "IYH", "VHT", "XBI", "IHI",
                    "XLF", "IYG", "VFH", "KIE",
                    "XLE", "IYE", "VDE", "XOP"]
kt_daily_df = Kendall_Tau(uni_daily_returns_df[names_l]).T
max_kt_daily = kt_daily_df.idxmax(axis=0)[0]
kt_daily_df.sort_values(by='KT', ascending=False, inplace=True)

print(kt_daily_df[:10])
print(f"Maximum KT value(daily) is {kt_daily_df['KT'][max_kt_daily]} in column '{max_kt_daily}'")

                 KT
IYE-VDE    0.946204
XLE-IYE    0.935025
XLE-VDE    0.932510
IYW-VITAX  0.919397
XLF-VFH    0.911629
VITAX-XLK  0.890166
XLF-IYG    0.879840
IYG-VFH    0.875434
IYW-XLK    0.872044
XLV-VHT    0.864693
Maximum KT value(daily) is 0.9462039299418779 in column 'IYE-VDE'


C:\Users\12451\AppData\Local\Temp/ipykernel_26372/1646480107.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  kendall_df[colname] = [tau]


## Selected ETFs
IYE-VDE -- iShares U.S. Energy ETF & Vanguard Energy Index Fund\
XLE-IYE -- Energy Select Sector SPDR Fund & iShares U.S. Energy ETF\
XLF-VFH -- Financial Select Sector SPDR Fund & Vanguard Financials Index Fund\
XLV-VHT -- Health Care Select Sector SPDR Fund & Vanguard Health Care Index Fund

# Use R to fit copula for each pair

In [184]:
index_pairs_l = [['IJH', 'MDY'], ['IYY', 'SPY'], ['OEF', 'QQQ'], ['IWM', 'IWO'], ['IYE', 'VDE'], ['XLE', 'IYE'],
                 ['XLF', 'VFH'], ['XLV', 'VHT']]

In [19]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr, data
pandas2ri.activate()

In [9]:
copula = importr('copula')
VineCopula = importr('VineCopula')
base = importr('base')
utils = importr('utils')

In [52]:
def bestCopula(pair_names):
    a1 = pd.DataFrame(uni_daily_returns_df[pair_names[0]])
    a2 = pd.DataFrame(uni_daily_returns_df[pair_names[1]])
    a1r = ro.conversion.py2rpy(a1)
    a2r = ro.conversion.py2rpy(a2)
    a1_pobs = copula.pobs(a1r)
    a2_pobs = copula.pobs(a2r)
    selectCopula = VineCopula.BiCopSelect(a1_pobs, a2_pobs)
    base.summary(selectCopula)
    return 0

def fitTCopula(pair_names):
    a1 = pd.DataFrame(uni_daily_returns_df[pair_names[0]])
    a2 = pd.DataFrame(uni_daily_returns_df[pair_names[1]])
    a1r = ro.conversion.py2rpy(a1)
    a2r = ro.conversion.py2rpy(a2)
    a1_pobs = copula.pobs(a1r)
    a2_pobs = copula.pobs(a2r)
    a1_pobs = a1_pobs.reshape(len(a1),)
    a2_pobs = a2_pobs.reshape(len(a2),)
    
    temp_df = pd.DataFrame(list(zip(a1_pobs, a1_pobs)),
                 columns=['a1','a12'],index=list(range(len(a1_pobs))) )
    student_t_cop = copula.tCopula(dim = 2)
    result = ro.r['fitCopula'](copula = student_t_cop, data = temp_df,\
                               method = 'ml', **{'optim.method': 'BFGS'})
    return result

In [185]:
for i in range(len(index_pairs_l)):
    bestCopula(index_pairs_l[i])

Family
------ 
No:    2
Name:  t

Parameter(s)
------------
par:  1
par2: 2
Dependence measures
-------------------
Kendall's tau:    0.96 (empirical = 0.95, p value < 0.01)
Upper TD:         0.96 
Lower TD:         0.96 

Fit statistics
--------------
logLik:  11410.79 
AIC:    -22817.59 
BIC:    -22804.95 

Family
------ 
No:    2
Name:  t

Parameter(s)
------------
par:  0.99
par2: 2
Dependence measures
-------------------
Kendall's tau:    0.93 (empirical = 0.92, p value < 0.01)
Upper TD:         0.93 
Lower TD:         0.93 

Fit statistics
--------------
logLik:  9376.44 
AIC:    -18748.87 
BIC:    -18736.23 

Family
------ 
No:    2
Name:  t

Parameter(s)
------------
par:  0.99
par2: 2
Dependence measures
-------------------
Kendall's tau:    0.89 (empirical = 0.89, p value < 0.01)
Upper TD:         0.89 
Lower TD:         0.89 

Fit statistics
--------------
logLik:  7634.8 
AIC:    -15265.6 
BIC:    -15252.96 

Family
------ 
No:    2
Name:  t

Parameter(s)
------------
par: 

In [266]:
kentall_tau_l = [0.96, 0.93, 0.89, 0.87, 0.95, 0.93, 0.91, 0.86]

We have 9 pairs and all of their return distributions fit to student-t copula the best.

In [97]:
def tCopProbs(pair_names):
    student_t_cop = copula.tCopula(dim = 2)
    a1 = pd.DataFrame(uni_daily_returns_df[pair_names[0]])
    a2 = pd.DataFrame(uni_daily_returns_df[pair_names[1]])
    a1r = ro.conversion.py2rpy(a1)
    a2r = ro.conversion.py2rpy(a2)
    a1_pobs = copula.pobs(a1r)
    a2_pobs = copula.pobs(a2r)
    a1_pobs = a1_pobs.reshape(len(a1),)
    a2_pobs = a2_pobs.reshape(len(a2),)

    temp_df = pd.DataFrame(list(zip(a1_pobs, a2_pobs)),
                 columns=['a1','a2'],index=list(range(len(a1_pobs))) )

    cop = ro.r['BiCopEst'](a1_pobs, a2_pobs, family = 2)
    temp = ro.r['BiCopHfunc'](a1_pobs, a2_pobs, cop)
    result = pd.DataFrame(temp).T
    return result 

In [186]:
pairs_probs_l = []
for i in index_pairs_l:
    pairs_probs_l.append(tCopProbs(i))

# Get Real Time Data

In [111]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper  
from ibapi.contract import Contract
from ibapi.order import *
import threading
import time
import math

In [161]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.returns_l = []
    
    def historicalData(self, reqId, bar):
        daily_r = bar.close/bar.open - 1
        self.returns_l.append(daily_r)
        print("Added a return")
    
    def historicalDataEnd(self, reqId, start, end):
        print("Successfully got data of reqID " + str(reqId))
        
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)
   
    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)
 
    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        #print(f"orderId: {orderId}, status: {status}, filled: {filled}, remaining: {remaining}, avgFillPrice: {avgFillPrice}, permId: {permId}, parentId: {parentId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice: {mktCapPrice}")
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
    
    def execDetails(self, reqId, contract, execution):
        #print(f"reqId: {reqId}, contract: {contract}, execution: {execution}")
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)

    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType == 2 and reqId == 1:
            self.ask1.append(price)
            #print('The last ask price for IJH is: ', self.ask1[-1])
            try:
                self.r1.append( self.ask1[-1] / self.ask1[-2])
            except:
                pass
            try:
                print('The return for IJH is', self.r1[-1])
            except:
                pass
        
        if tickType == 2 and reqId == 2:
            self.ask2.append(price)
            #print('The last ask price for MDY is: ', self.ask1[-1])
            try:
                self.r2.append( self.ask2[-1] / self.ask2[-2])
            except:
                pass
            try:
                print('The return for MDY is', self.r2[-1])
            except:
                pass
            
    def Stock_contract(self, symbol, secType='STK', exchange='SMART', currency='USD'):
        ''' custom function to create contract '''
        contract = Contract()
        contract.symbol = symbol
        contract.secType = secType
        contract.exchange = exchange
        contract.currency = currency
        return contract
        
    def error_handler(msg):
        print(msg)
        
def submit_order(contract, direction, qty=100, ordertype='MKT', transmit=True):
    #Create order object
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = ordertype
    order.transmit = transmit
    #submit order
    app.placeOrder(app.nextorderId, contract, order)
    app.nextorderId += 1

def MKTorder(action,n):
    order = Order()
    order.action = action
    order.totalQuantity = n
    order.orderType = 'MKT'
    order.eTradeOnly = False
    order.firmQuoteOnly = False
    return order
    
def run_loop():
    app.run()


In [110]:
# Contracts
def stockContract(tag):
    contract = Contract()
    contract.symbol = tag
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    
    return contract

In [188]:
contract_l = []
for i in index_pairs_l:
    for j in range(2):
        contract_l.append(stockContract(i[j]))

In [189]:
contract_l

[1690452288224: 0,IJH,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690452288320: 0,MDY,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690452289184: 0,IYY,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690452291488: 0,SPY,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690452288416: 0,OEF,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690452290240: 0,QQQ,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690452290384: 0,IWM,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690452290960: 0,IWO,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450143984: 0,IYE,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450142784: 0,VDE,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450143456: 0,XLE,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450145088: 0,IYE,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450144176: 0,XLF,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450144800: 0,VFH,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450142544: 0,XLV,STK,,0.0,,,SMART,,USD,,,False,,combo:,
 1690450142208: 0,VHT,STK,,0.0,,,SMART,,USD,,,False,,combo:]

In [229]:
# stock tags in a simple list
tags_l = [element for sublist in index_pairs_l for element in sublist]

# Historical Returns
returns = uni_daily_returns_df[tags_l]
# IJH_min_r and MDY_min_r, both np.array

In [223]:
# Establish Connection 
app = IBapi()
app.connect('127.0.0.1', 7450, 999)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

ERROR -1 2104 Market data farm connection is OK:usbond
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  165


In [224]:
reqId = 1

In [225]:
# Request today's closing price and update the prices to the return dataframe
for i in range(len(contract_l)):
    app.reqHistoricalData(reqId, contract_l[i], "", "1 D", "1 day", "MIDPOINT", 0, 1, False, [])
    reqId += 1
    time.sleep(1)

Added a return
Successfully got data of reqID 1
Added a return
Successfully got data of reqID 2
Added a return
Successfully got data of reqID 3
Added a return
Successfully got data of reqID 4
Added a return
Successfully got data of reqID 5
Added a return
Successfully got data of reqID 6
Added a return
Successfully got data of reqID 7
Added a return
Successfully got data of reqID 8
Added a return
Successfully got data of reqID 9
Added a return
Successfully got data of reqID 10
Added a return
Successfully got data of reqID 11
Added a return
Successfully got data of reqID 12
Added a return
Successfully got data of reqID 13
Added a return
Successfully got data of reqID 14
Added a return
Successfully got data of reqID 15
Added a return
Successfully got data of reqID 16


In [226]:
len(app.returns_l) - len(contract_l)

0

In [227]:
new_returns = app.returns_l[:]

In [222]:
app.disconnect()

In [237]:
# Update the new returns
today = date.today()
returns.loc[str(today)] = new_returns

C:\Users\12451\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


# Calculate conditional probabilities

In [263]:
pairs_probs_l = []
for i in index_pairs_l:
    temp = tCopProbs(i).tail(1)
    pairs_probs_l.append(np.array(temp)[0])

In [264]:
pairs_probs_l

[array([0.53488732, 0.46206725]),
 array([0.40418394, 0.58965829]),
 array([0.86237597, 0.13616385]),
 array([0.48469557, 0.50647605]),
 array([0.85171589, 0.1733772 ]),
 array([0.28054229, 0.75752789]),
 array([0.43550167, 0.54483422]),
 array([0.24217436, 0.80075749])]

In [265]:
index_pairs_l

[['IJH', 'MDY'],
 ['IYY', 'SPY'],
 ['OEF', 'QQQ'],
 ['IWM', 'IWO'],
 ['IYE', 'VDE'],
 ['XLE', 'IYE'],
 ['XLF', 'VFH'],
 ['XLV', 'VHT']]

# Trade

In [282]:
shares = [[200, 109], [498, 121], [264, 156], [288, 225], [1138, 434], [595, 1138.17], [1526, 636], [378, 243.56]]

In [297]:
# Establish Connection 
app = IBapi()
app.connect('127.0.0.1', 7450, 999)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm


The next valid order id is:  165


ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [298]:
for i in range(0, len(pairs_probs_l), 2):
    hedge_ratio = kentall_tau_l[i]
    if(pairs_probs_l[i][0] <= 0.21 and pairs_probs_l[i+1][1] >= 0.79):
        # Long the first ETF in of the pair and short the second ETF
        app.placeOrder(app.nextorderId, contract_l[i], MKTorder('BUY', shares[i/2][0]))
        app.nextorderId += 1
        app.placeOrder(app.nextorderId, contract_l[i+1], MKTorder('SELL', shares[i/2][1]))
        app.nextorderId += 1
    elif(pairs_probs_l[i][0] >= 0.79 and pairs_probs_l[i+1][1] <= 0.21):
        # Short the first ETF, long the second
        app.placeOrder(app.nextorderId, contract_l[i], MKTorder('SELL', shares[i/2][0]))
        app.nextorderId += 1
        app.placeOrder(app.nextorderId, contract_l[i+1], MKTorder('BUY', shares[i/2][1]))
        app.nextorderId += 1
print("Trade Ended")
app.disconnect()

Trade Ended


In [296]:
app.disconnect()